In [231]:
import numpy as np
import sympy as sm
import sympy.physics.mechanics as me
from sympy.physics.vector import init_vprinting, vlatex
from pydy.codegen.ode_function_generators import generate_ode_function
from scipy.integrate import odeint
from pydy.viz.shapes import Cylinder, Sphere
import pydy.viz
pydy.viz.shapes.__all__
from pydy.viz.visualization_frame import VisualizationFrame
from pydy.viz.scene import Scene

**Reference Frames:** Create reference frames for all of the bodies     

In [232]:
world_frame = me.ReferenceFrame('W')
pelvis_frame = me.ReferenceFrame('P')
torso_frame = me.ReferenceFrame('T')
right_thigh_frame = me.ReferenceFrame('RT')
left_thigh_frame = me.ReferenceFrame('LT')
right_shank_frame = me.ReferenceFrame('RS')
left_shank_frame = me.ReferenceFrame('LS')
right_foot_frame = me.ReferenceFrame('RF')
left_foot_frame = me.ReferenceFrame('LF')

**CoM Points:** Create points for the CoM of each body

In [233]:
origin = me.Point('O')
pelvis_mass_center = me.Point('P_CoM')
torso_mass_center = me.Point('T_CoM')
right_thigh_mass_center = me.Point('RT_CoM')
left_thigh_mass_center = me.Point('LT_CoM')
right_shank_mass_center = me.Point('RS_CoM')
left_shank_mass_center = me.Point('LS_CoM')
right_foot_mass_center = me.Point('RF_CoM')
left_foot_mass_center = me.Point('LF_CoM')

**Generalized Coordinates and Speeds:**

In [234]:
# Create the generalized coordinates
qPx, qPy, qPz, qL, qRH, qRK, qRA, qLH, qLK, qLA = me.dynamicsymbols('qPx qPy qPz qL qRH qRK qRA qLH qLK qLA')
# Create the generalized speeds
vPx, vPy, vPz, vL, vRH, vRK, vRA, vLH, vLK, vLA = me.dynamicsymbols('vPx vPy vPz vL vRH vRK vRA vLH vLK vLA')

**Constants:** Define the required constants

In [235]:
#Mass
pelvis_mass, torso_mass, thigh_mass, shank_mass, foot_mass = sm.symbols('m_pelvis m_torso m_thigh m_shank m_foot')
#Moment of Inertia
pelvis_inertia, torso_inertia, thigh_inertia, shank_inertia, foot_inertia = sm.symbols('I_pelvis I_torso I_thigh I_shank I_foot')
#Declare the necessary length of each segment
pelvis_length, torso_length, thigh_length, shank_length, foot_length = sm.symbols('L_pelvis L_torso L_thigh L_shank L_foot')
#Declare the width of the foot
foot_width = sm.symbols('W_foot')
#Height of the CoM - relative to the bottom end of the segment
pelvis_com_length, torso_com_length, thigh_com_length, shank_com_length, foot_com_length = sm.symbols('c_pelvis c_torso c_thigh c_shank c_foot')
#Forward position of the foot CoM
foot_com_length_x = sm.symbols('cx_foot')

**System:** Declare the system

In [236]:
system = me.System(world_frame,origin)

**Rigid Bodies:** Define the rigid bodies

In [237]:
world = me.RigidBody('World',
                     masscenter=origin,
                     frame = world_frame)

pelvis = me.RigidBody('Pelvis',
                      masscenter = pelvis_mass_center,
                      frame = pelvis_frame,
                      mass = pelvis_mass,
                      inertia = (me.inertia(pelvis_frame,
                                            ixx = 0,
                                            iyy = 0,
                                            izz = pelvis_inertia,
                                            ixy = 0,
                                            iyz = 0,
                                            izx = 0),
                                            pelvis_mass_center))

torso = me.RigidBody('Torso',
                     masscenter = torso_mass_center,
                     frame = torso_frame,
                     mass = torso_mass,
                     inertia = (me.inertia(torso_frame,
                                           ixx = 0,
                                           iyy = 0,
                                           izz = torso_inertia,
                                           ixy = 0,
                                           iyz = 0,
                                           izx = 0,),
                                           torso_mass_center))

right_thigh = me.RigidBody('Right Thigh',
                           masscenter = right_thigh_mass_center,
                           frame = right_thigh_frame,
                           mass = thigh_mass,
                           inertia = (me.inertia(right_thigh_frame,
                                                 ixx = 0,
                                                 iyy = 0,
                                                 izz = thigh_inertia,
                                                 ixy = 0,
                                                 iyz = 0,
                                                 izx = 0),
                                                 right_thigh_mass_center))

left_thigh = me.RigidBody('Left Thigh',
                           masscenter = left_thigh_mass_center,
                           frame = left_thigh_frame,
                           mass = thigh_mass,
                           inertia = (me.inertia(left_thigh_frame,
                                                 ixx = 0,
                                                 iyy = 0,
                                                 izz = thigh_inertia,
                                                 ixy = 0,
                                                 iyz = 0,
                                                 izx = 0),
                                                 left_thigh_mass_center))

right_shank = me.RigidBody('Right Shank',
                           masscenter = right_shank_mass_center,
                           frame = right_shank_frame,
                           mass = shank_mass,
                           inertia = (me.inertia(right_shank_frame,
                                                 ixx = 0,
                                                 iyy = 0,
                                                 izz = shank_inertia,
                                                 ixy = 0,
                                                 iyz = 0,
                                                 izx = 0),
                                                 right_shank_mass_center))

left_shank = me.RigidBody('Left Shank',
                           masscenter = left_shank_mass_center,
                           frame = left_shank_frame,
                           mass = shank_mass,
                           inertia = (me.inertia(left_shank_frame,
                                                 ixx = 0,
                                                 iyy = 0,
                                                 izz = shank_inertia,
                                                 ixy = 0,
                                                 iyz = 0,
                                                 izx = 0),
                                                 left_shank_mass_center))

right_foot = me.RigidBody('Right Foot',
                           masscenter = right_foot_mass_center,
                           frame = right_foot_frame,
                           mass = foot_mass,
                           inertia = (me.inertia(right_foot_frame,
                                                 ixx = 0,
                                                 iyy = 0,
                                                 izz = foot_inertia,
                                                 ixy = 0,
                                                 iyz = 0,
                                                 izx = 0),
                                                 right_foot_mass_center))

left_foot = me.RigidBody('Left Foot',
                           masscenter = left_foot_mass_center,
                           frame = left_foot_frame,
                           mass = foot_mass,
                           inertia = (me.inertia(left_foot_frame,
                                                 ixx = 0,
                                                 iyy = 0,
                                                 izz = foot_inertia,
                                                 ixy = 0,
                                                 iyz = 0,
                                                 izx = 0),
                                                 left_foot_mass_center))

**Add Bodies:** Add the bodies to the system

In [238]:
system.add_bodies(world,pelvis,torso,
                  right_thigh,left_thigh,
                  right_shank,left_shank,
                  right_foot,left_foot)

**Joints:** Define the joints between the bodies

In [239]:
slider_Px = me.PrismaticJoint('Pelvis X',
                              parent = world,
                              child  = pelvis,
                              coordinates=qPx,
                              speeds = vPx,
                              joint_axis=world_frame.x)

slider_Py = me.PrismaticJoint('Pelvis Y',
                              parent = world,
                              child  = pelvis,
                              coordinates=qPy,
                              speeds = vPy,
                              joint_axis=world_frame.y)

revolute_Pz = me.PinJoint('Pelvis Z',
                          parent = world,
                          child = pelvis,
                          coordinates=qPz,
                          speeds = vPz,
                          joint_axis=world_frame.z)

lumbar = me.PinJoint('Lumbar',
                     parent = pelvis,
                     child = torso,
                     coordinates = qL,
                     speeds = vL,
                     parent_point = (pelvis_length-pelvis_com_length)*pelvis_frame.y,
                     child_point = -torso_com_length*torso_frame.y)

right_hip = me.PinJoint('Right Hip',
                        parent = pelvis,
                        child = right_thigh,
                        coordinates = qRH,
                        speeds = vRH,
                        parent_point = -pelvis_com_length*pelvis_frame.y,
                        child_point = (thigh_length-thigh_com_length)*right_thigh_frame.y)

left_hip = me.PinJoint('Left Hip',
                        parent = pelvis,
                        child = left_thigh,
                        coordinates = qLH,
                        speeds = vLH,
                        parent_point = -pelvis_com_length*pelvis_frame.y,
                        child_point = (thigh_length-thigh_com_length)*left_thigh_frame.y)

right_knee = me.PinJoint('Right Knee',
                         parent = right_thigh,
                         child = right_shank,
                         coordinates = qRK,
                         speeds = vRK,
                         parent_point = -thigh_com_length*right_thigh_frame.y,
                         child_point = (shank_length-shank_com_length)*right_shank.y)

left_knee = me.PinJoint('Left Knee',
                         parent = left_thigh,
                         child = left_shank,
                         coordinates = qLK,
                         speeds = vLK,
                         parent_point = -thigh_com_length*left_thigh_frame.y,
                         child_point = (shank_length-shank_com_length)*left_shank.y)

right_ankle = me.PinJoint('Right Ankle',
                          parent = right_shank,
                          child = right_foot,
                          coordinates = qRA,
                          speeds = vRA,
                          parent_point = -shank_com_length*right_shank_frame.y,
                          child_point = (foot_length-foot_com_length)*right_foot_frame.y - foot_com_length_x*right_foot_frame.x)

left_ankle = me.PinJoint('Left Ankle',
                          parent = left_shank,
                          child = left_foot,
                          coordinates = qLA,
                          speeds = vLA,
                          parent_point = -shank_com_length*left_shank_frame.y,
                          child_point = (foot_length-foot_com_length)*left_foot_frame.y - foot_com_length_x*left_foot_frame.x)

**Add Joints:** Add the joints to the system

In [240]:
system.add_joints(slider_Px,slider_Py,
                  revolute_Pz, lumbar,
                  right_hip,left_hip,
                  right_knee, left_knee,
                  right_ankle,left_ankle)

**Joint Actuators:** Add actuators to the joints

In [ ]:
# Create generalized forces
uL, uRH, uRK, uRA, uLH, uLK, uLA = me.dynamicsymbols('uL uRH uRK uRA uLH uLK uLA')


system.add_actuators(me.TorqueActuator(uL,pelvis_frame.z,torso_frame,pelvis_frame),
                     me.TorqueActuator(uRH,pelvis.z,pelvis,right_thigh),
                     me.TorqueActuator(uLH,pelvis.z,pelvis,left_thigh),
                     me.TorqueActuator(uRK,right_thigh.z,right_thigh,right_shank),
                     me.TorqueActuator(uLK,left_thigh.z,left_thigh,left_shank),
                     me.TorqueActuator(uRA,right_shank.z,right_shank,right_foot),
                     me.TorqueActuator(uLA,left_shank.z,left_shank,left_foot))

#Gravity
g = sm.symbols('g')
system.apply_uniform_gravity(-g*world_frame.y)

**Equations of Motion:** Use Kane's method to find mass matrix and forcing vector

In [242]:
system.form_eoms(explicit_kinematics = True)
mass_matrix = system.mass_matrix_full
forcing_vector = system.forcing_full

**Generate ODE:** Use PyDy to generate the ODE

In [243]:
constants = [pelvis_length,
             torso_length,
             thigh_length,
             shank_length,
             foot_length,
             foot_width,
             pelvis_com_length,
             torso_com_length,
             thigh_com_length,
             shank_com_length,
             foot_com_length,
             foot_com_length_x,
             pelvis_mass,
             torso_mass,
             thigh_mass,
             shank_mass,
             foot_mass,
             pelvis_inertia,
             torso_inertia,
             thigh_inertia,
             shank_inertia,
             foot_inertia,
             g]


coordinates = [qPx,qPy,qPz,qL,qRH,qLH,qRK,qLK,qRA,qLA]
speeds = [vPx,vPy,vPz,vL,vRH,vLH,vRK,vLK,vRA,vLA]
specified = [uL, uRH, uRK, uRA, uLH, uLK, uLA]


right_hand_side = generate_ode_function(forcing_vector,
                                        coordinates,speeds,
                                        constants,
                                        mass_matrix = mass_matrix,
                                        specifieds = specified,
                                        generator = 'cython')

help(right_hand_side)


Help on function rhs in module pydy.codegen.ode_function_generators:

rhs(*args)
    Returns the derivatives of the states, i.e. numerically evaluates the right
    hand side of the first order differential equation.

    x' = f(x, t, r, p)

    Parameters
    x : ndarray, shape(20,)
        The state vector is ordered as such:
            - qPx(t)
            - qPy(t)
            - qPz(t)
            - qL(t)
            - qRH(t)
            - qLH(t)
            - qRK(t)
            - qLK(t)
            - qRA(t)
            - qLA(t)
            - vPx(t)
            - vPy(t)
            - vPz(t)
            - vL(t)
            - vRH(t)
            - vLH(t)
            - vRK(t)
            - vLK(t)
            - vRA(t)
            - vLA(t)
    t : float
        The current time.
    r : dictionary; ndarray, shape(7,); function

        There are three options for this argument. (1) is more flexible but
        (2) and (3) are much more efficient.

        (1) A dictionary that maps the s

**Function Inputs:** Define the inputs the the function

In [244]:
#Initial Conditions
x0 = np.zeros(20)

#Constants
numerical_constants = np.array([0.1, #Pevlis length [m]
                                1.0, #torso length [m]
                                0.6, #thigh length [m]
                                0.5, #shank length [m]
                                0.1, #foot length
                                0.3, #Foot width
                                0.05, #Pelvis com length [m]
                                0.5, #torso com length [m]
                                0.3, #thigh com length 
                                0.25, #shank com length 
                                0.05, #foot com length
                                0.15, #foot com length x
                                10.0, #pelvis mass
                                30.0, #torso mass 
                                15.0, #Thigh mass
                                10.0, #shank mass
                                5.0, #foot mass
                                3.0, #Pelvis inertia
                                8.0, #torso inertia
                                2.0, #thigh inertia
                                1.0, #shank inertia
                                0.5,
                                9.81]) #Foot inertia

numerical_specified = np.zeros(7)

args = {'constants': numerical_constants,
        'specified': numerical_specified}


frame_per_sec = 60.0
final_time = 10.0

t = np.linspace(0.0,final_time,int(final_time*frame_per_sec))

**Integration:** Integrate the ODE

In [245]:
y = odeint(right_hand_side,x0,t,args=(numerical_specified,numerical_constants))

LinAlgError: Singular matrix